In [1]:
import pandas as pd
import os


folder_path = 'anova'

files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]  # или .xlsx, в зависимости от формата файлов

# Словарь для хранения результатов
results = {}

# Обработка каждого файла
for file in files:
    # Чтение данных из файла
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)  # или pd.read_excel(file_path), если файлы в формате Excel

    # Убедитесь, что переменные condition и slider_2.response существуют
    if 'condition' in data.columns and 'slider_2.response' in data.columns:
        # Группировка данных по категории и вычисление среднего
        averages = data.groupby('condition')['slider_2.response'].mean()

        # Сохранение результатов в словарь
        respondent_id = file.split('.')[0]  # Предполагаем, что имя файла - это ID респондента
        results[respondent_id] = {
            'emot': averages.get('emot', None),
            'less': averages.get('less', None),
            'filler': averages.get('filler', None)
        }

# Преобразование результатов в DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index').reset_index()
results_df.columns = ['Respondent', 'Emot', 'Less', 'Filler']

# Сохранение в Excel файл
results_df.to_excel('averages_by_category.xlsx', index=False)

In [3]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Загрузка данных из Excel файла
data = pd.read_excel('averages_by_category.xlsx')

# Преобразование данных в длинный формат для ANOVA
data_melted = data.melt(id_vars=['Respondent'], value_vars=['Emot', 'Less', 'Filler'],
                         var_name='Condition', value_name='Score')

# Выполнение однофакторного дисперсионного анализа
model = ols('Score ~ Condition', data=data_melted).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Вывод результатов ANOVA
print("ANOVA Results:")
print(anova_table)

# Расчет размера эффекта η²
ss_total = anova_table['sum_sq'].sum()
ss_between = anova_table['sum_sq']['Condition']
eta_squared = ss_between / ss_total

print("\nEffect Size (η²):", eta_squared)

# Интерпретация результатов
if anova_table['PR(>F)']['Condition'] < 0.05:
    print("\nРезультаты ANOVA значимы: существует статистически значимая разница между группами.")
else:
    print("\nРезультаты ANOVA незначимы: нет статистически значимой разницы между группами.")

ANOVA Results:
               sum_sq     df           F        PR(>F)
Condition  248.583651    2.0  444.072808  1.086495e-53
Residual    31.067862  111.0         NaN           NaN

Effect Size (η²): 0.8889050822887414

Результаты ANOVA значимы: существует статистически значимая разница между группами.


In [7]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Загрузка данных из Excel файла
data = pd.read_excel('averages_by_category.xlsx')

# Преобразование данных в длинный формат для ANOVA
data_melted = data.melt(id_vars=['Respondent'], value_vars=['Emot', 'Less', 'Filler'],
                         var_name='Condition', value_name='Score')

# Выполнение однофакторного дисперсионного анализа (One-Way ANOVA)
model = ols('Score ~ Condition', data=data_melted).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Вывод результатов ANOVA
print("ANOVA Results:")
print(anova_table)

# Расчет размера эффекта η²
ss_total = anova_table['sum_sq'].sum()
ss_between = anova_table['sum_sq']['Condition']
ss_within = anova_table['sum_sq']['Residual']

# Расчет частичного η²
eta_partial_squared = ss_between / (ss_between + ss_within)

print("\nPartial Effect Size (ηp²):", eta_partial_squared)

# Интерпретация результатов
if anova_table['PR(>F)']['Condition'] < 0.05:
    print("\nРезультаты One-Way ANOVA значимы: существует статистически значимая разница между группами.")
else:
    print("\nРезультаты One-Way ANOVA незначимы: нет статистически значимой разницы между группами.")

ANOVA Results:
               sum_sq     df           F        PR(>F)
Condition  248.583651    2.0  444.072808  1.086495e-53
Residual    31.067862  111.0         NaN           NaN

Partial Effect Size (ηp²): 0.8889050822887414

Результаты One-Way ANOVA значимы: существует статистически значимая разница между группами.


In [11]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# Загрузка данных из файла
data = pd.read_excel('averages_by_category.xlsx')

# Преобразуем данные в длинный формат
data_long = pd.melt(data, id_vars=['Respondent'], value_vars=['Emot', 'Less', 'Filler'],
                    var_name='Category', value_name='Score')

# Выполнение однофакторного дисперсионного анализа
model = ols('Score ~ C(Category)', data=data_long).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Вывод результатов ANOVA
print(anova_table)

# Расчет частичного размера эффекта ηp²
ss_total = sum((data_long['Score'] - data_long['Score'].mean())**2)
ss_between = anova_table['sum_sq']['C(Category)']
ss_within = anova_table['sum_sq']['Residual']

eta_squared = ss_between / ss_total
eta_partial_squared = ss_between / (ss_between + ss_within)

print(f"Размер эффекта η²: {eta_squared:.4f}")
print(f"Частичный размер эффекта ηp²: {eta_partial_squared:.4f}")

# Интерпретация результата
if eta_partial_squared < 0.01:
    effect_size_interpretation = "Малый эффект"
elif eta_partial_squared < 0.06:
    effect_size_interpretation = "Средний эффект"
else:
    effect_size_interpretation = "Большой эффект"

print(f"Интерпретация размера эффекта: {effect_size_interpretation}")

                 sum_sq     df           F        PR(>F)
C(Category)  248.583651    2.0  444.072808  1.086495e-53
Residual      31.067862  111.0         NaN           NaN
Размер эффекта η²: 0.8889
Частичный размер эффекта ηp²: 0.8889
Интерпретация размера эффекта: Большой эффект


In [9]:
import pandas as pd
from scipy import stats

# Загрузка данных из Excel-файла
file_path = 'averages_by_category1.xlsx'
data = pd.read_excel(file_path)

# Переменные для проверки
variables = ['Emot1', 'Less1', 'Filler1', 'Emot', 'Less', 'Filler']

# Проверка нормальности для каждой переменной
results = {}
for var in variables:
    if var in data.columns:
        stat, p_value = stats.shapiro(data[var].dropna())  # Удаляем NaN значения
        results[var] = {
            'statistic': stat,
            'p_value': p_value,
            'is_normal': p_value > 0.05  # Нулевая гипотеза о нормальности
        }
    else:
        results[var] = 'Variable not found in data'

# Вывод результатов
for var, result in results.items():
    if isinstance(result, dict):
        print(f"{var}: Statistic={result['statistic']}, p-value={result['p_value']}, is normal={result['is_normal']}")
    else:
        print(f"{var}: {result}")

Emot1: Statistic=0.9312158823013306, p-value=0.02191239222884178, is normal=False
Less1: Statistic=0.9529996514320374, p-value=0.11183279007673264, is normal=True
Filler1: Statistic=0.9154956340789795, p-value=0.007190981414169073, is normal=False
Emot: Statistic=0.9312157034873962, p-value=0.0219121053814888, is normal=False
Less: Statistic=0.952999472618103, p-value=0.11183127015829086, is normal=True
Filler: Statistic=0.9154956340789795, p-value=0.007190981414169073, is normal=False


In [17]:
import pandas as pd
from scipy.stats import mannwhitneyu

# Загрузка данных
data = pd.read_csv('0.csv')

# Переменные для анализа
variables = ['Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust']
#speakers = ['PROC', 'SHUC']
attitudes = ['pro', 'anti']
results = []

# Разделение данных по Attitude и выполнение теста Манна-Уитни
for attitude in attitudes:
    #group_data = data[data['Speaker'] == speaker]
    group_data = data[data['Attitude'] == attitude]
    
    for variable in variables:
        pro_values = group_data[group_data['Speaker'] == 'PROC'][variable]
        anti_values = group_data[group_data['Speaker'] == 'SHUC'][variable]
        
        # Выполнение теста Манна-Уитни
        stat, p_value = mannwhitneyu(pro_values, anti_values)
        
        # Сохранение результатов
        results.append({
            'Attitude': attitude,
            'Variable': variable,
            'Statistic': stat,
            'p-value': p_value
        })

# Создание DataFrame с результатами
results_df = pd.DataFrame(results)

# Вывод результатов
print(results_df)

# Сохранение результатов в Excel
results_df.to_excel('11.xlsx', index=False)

  Attitude           Variable  Statistic       p-value
0      pro     Persuasiveness      656.0  1.219334e-02
1      pro            Sharing      689.0  2.848530e-03
2      pro         Congruence      634.0  2.813865e-02
3      pro  Authoritativeness      769.0  2.018177e-05
4      pro              Trust      521.5  5.643605e-01
5     anti     Persuasiveness      780.5  6.438364e-07
6     anti            Sharing      730.0  2.129929e-05
7     anti         Congruence      703.0  1.425219e-04
8     anti  Authoritativeness      782.0  3.883092e-07
9     anti              Trust      704.0  1.341404e-04


In [29]:
import pandas as pd
import statsmodels.api as sm

# Загрузка данных
data = pd.read_csv('01.csv')

# Замена запятых на точки во всех переменных
data = data.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)

# Преобразование категориальной переменной Attitude в числовую
data['Attitude'] = data['Attitude'].map({'pro': 1, 'anti': 0})

# Преобразование типов данных в числовые
columns_to_convert = ['CRT', 'NFCS', 'CS', 'Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust', '50_75_diff_AMP']
data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Удаление строк с NaN значениями
data = data.dropna(subset=columns_to_convert + ['Attitude'])

# Определение независимых переменных и зависимой переменной
X = data[columns_to_convert]
y = data['Attitude']

# Добавление константы для логистической регрессии
X = sm.add_constant(X)

# Подходящая логистическая регрессия
model = sm.Logit(y, X)
result = model.fit()

# Получение результатов
summary = result.summary2().tables[1]

# Сохранение результатов в Excel
summary.to_excel('13.xlsx')

print("Результаты логистической регрессии сохранены в файл '12.xlsx'.")

Optimization terminated successfully.
         Current function value: 0.652392
         Iterations 5
Результаты логистической регрессии сохранены в файл '12.xlsx'.


/var/folders/tg/kkh8vd7s0kzdck7rptzdy4sh0000gn/T/ipykernel_36729/1197624218.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)


In [33]:
import pandas as pd
import statsmodels.api as sm

# Загрузка данных
data = pd.read_csv('01.csv')

# Замена запятых на точки во всех переменных
data = data.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)

# Преобразование типов данных в числовые
columns_to_convert = ['CRT', 'NFCS', 'CS', 'Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust', '50_75_diff_AMP']
data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Удаление строк с NaN значениями
data = data.dropna(subset=columns_to_convert)

# Определение независимых переменных и зависимой переменной
X = data[['CRT', 'NFCS', 'CS', 'Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust']]
y = data['50_75_diff_AMP']

# Добавление константы для регрессии
X = sm.add_constant(X)

# Выполнение робастной регрессии
model = sm.RLM(y, X)
result = model.fit()

# Получение результатов
summary = result.summary()

# Сохранение результатов в Excel
with pd.ExcelWriter('15.xlsx') as writer:
    summary_df = pd.DataFrame(summary.tables[1])
    summary_df.to_excel(writer, sheet_name='Results')

print("Результаты робастной регрессии сохранены в файл 'robust_regression_results.xlsx'.")

Результаты робастной регрессии сохранены в файл 'robust_regression_results.xlsx'.


/var/folders/tg/kkh8vd7s0kzdck7rptzdy4sh0000gn/T/ipykernel_36729/477740546.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)


In [3]:
import pandas as pd
from sklearn.decomposition import FactorAnalysis
import numpy as np

# Загрузка данных из CSV файла
data = pd.read_csv('01.csv')

# Выбор переменных для анализа
variables = ['Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust']
X = data[variables]

# Выполнение эксплораторного факторного анализа
fa = FactorAnalysis(n_components=2)
X_factors = fa.fit_transform(X)

# Получение нагрузок факторов
loadings = fa.components_.T

# Создание DataFrame для нагрузок
loadings_df = pd.DataFrame(loadings, columns=['Factor 1', 'Factor 2'], index=variables)

# Сохранение результатов в Excel
with pd.ExcelWriter('18.xlsx') as writer:
    loadings_df.to_excel(writer, sheet_name='Loadings')
    pd.DataFrame(X_factors, columns=['Factor 1', 'Factor 2']).to_excel(writer, sheet_name='Factor Scores')

print("Эксплораторный факторный анализ завершен. Результаты сохранены в 'factor_analysis_results.xlsx'.")

Эксплораторный факторный анализ завершен. Результаты сохранены в 'factor_analysis_results.xlsx'.


In [5]:
import pandas as pd
from sklearn.decomposition import FactorAnalysis

# Загрузка данных из CSV файла
data = pd.read_csv('01.csv')

# Выбор переменных для анализа
variables = ['Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust']
X = data[variables]

# Выполнение эксплораторного факторного анализа
fa = FactorAnalysis(n_components=2)
fa.fit(X)

# Получение нагрузок факторов
loadings = fa.components_.T

# Создание DataFrame для нагрузок
loadings_df = pd.DataFrame(loadings, columns=[f'Factor {i+1}' for i in range(loadings.shape[1])], index=variables)

# Вывод факторных нагрузок
print("Факторные нагрузки для каждой переменной:")
print(loadings_df)

# Сохранение результатов в Excel
loadings_df.to_excel('19.xlsx', sheet_name='Loadings')

print("Факторные нагрузки сохранены в 'factor_loadings.xlsx'.")

Факторные нагрузки для каждой переменной:
                   Factor 1  Factor 2
Persuasiveness    -1.708631  0.011267
Sharing           -1.346600 -0.162865
Congruence        -1.117790  0.162649
Authoritativeness -1.115964 -0.912870
Trust             -1.511839  0.545231
Факторные нагрузки сохранены в 'factor_loadings.xlsx'.


In [33]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
import numpy as np

# Загрузка данных из CSV файла
data = pd.read_csv('01_1.csv')

# Проверка на наличие пропущенных значений
print(data.isnull().sum())

# Выбор переменных для анализа
variables = ['Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust']
data_subset = data[variables]

# Проведение эксплораторного факторного анализа
fa = FactorAnalyzer(n_factors=2, rotation='varimax')
fa.fit(data_subset)

# Получение результатов
loadings = fa.loadings_
variance = fa.get_factor_variance()

# Создание DataFrame для загрузок факторов
loadings_df = pd.DataFrame(loadings, index=variables, columns=[f'Factor {i+1}' for i in range(2)])

# Подготовка результатов для записи в Excel
results = {
    'Factor Loadings': loadings_df,
    'Variance Explained': variance
}

# Запись результатов в Excel файл
with pd.ExcelWriter('20.xlsx') as writer:
    loadings_df.to_excel(writer, sheet_name='Factor Loadings')
    
    # Создание DataFrame для вариации
    variance_df = pd.DataFrame(variance, index=['Eigenvalue', 'Variance Explained', 'Cumulative Variance'])
    variance_df.to_excel(writer, sheet_name='Variance Explained')

print("Эксплораторный факторный анализ завершен. Результаты сохранены в 'factor_analysis_results.xlsx'.")

Unnamed: 0           0
SBJ_ID               0
Attitude             0
PROCENKO_AMP_E       0
PROCENKO_AMP_C       0
diff_amp             0
CRT                  0
NFCS                 0
CS                   0
Speaker              0
60_65_AMP_E          0
60_65_AMP_C          0
60_65_diff_AMP       0
50_75_AMP_E          0
50_75_AMP_C          0
50_75_diff_AMP       0
Persuasiveness       0
Sharing              0
Congruence           0
Authoritativeness    0
Trust                0
Factor 1             0
Factor 2             0
dtype: int64
Эксплораторный факторный анализ завершен. Результаты сохранены в 'factor_analysis_results.xlsx'.


In [37]:
import pandas as pd
from factor_analyzer import FactorAnalyzer

# Загрузка данных из CSV файла
data = pd.read_csv('01_12.csv')

# Проверка на наличие пропущенных значений
print(data.isnull().sum())

# Выбор переменных для анализа
variables = ['Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust']
data_subset = data[variables]

# Проведение эксплораторного факторного анализа
fa = FactorAnalyzer(n_factors=2, rotation='varimax')
fa.fit(data_subset)

# Получение факторных баллов
factor_scores = fa.transform(data_subset)

# Создание DataFrame для факторных баллов
factor_scores_df = pd.DataFrame(factor_scores, columns=[f'Factor {i+1}' for i in range(factor_scores.shape[1])])

# Объединение исходных данных с факторными баллами
data_with_factors = pd.concat([data, factor_scores_df], axis=1)

# Сохранение результатов в новый CSV файл
data_with_factors.to_csv('01.csv', index=False)

print("Факторные баллы добавлены. Результаты сохранены в '01_.csv'.")

Unnamed: 0           0
SBJ_ID               0
Attitude             0
PROCENKO_AMP_E       0
PROCENKO_AMP_C       0
diff_amp             0
CRT                  0
NFCS                 0
CS                   0
Speaker              0
60_65_AMP_E          0
60_65_AMP_C          0
60_65_diff_AMP       0
50_75_AMP_E          0
50_75_AMP_C          0
50_75_diff_AMP       0
Persuasiveness       0
Sharing              0
Congruence           0
Authoritativeness    0
Trust                0
dtype: int64
Факторные баллы добавлены. Результаты сохранены в '01_.csv'.


In [39]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder

def robust_regression(data_path):
    try:
        # Загрузка данных
        df = pd.read_csv(data_path)
        
        # Проверка наличия необходимых столбцов
        targets = ['Persuasiveness', 'Sharing', 'Congruence', 'Authoritativeness', 'Trust', 'Factor 1', 'Factor 2']
        required_columns = ['Attitude', 'Speaker'] + targets
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"Column '{col}' is missing in the data.")
        
        # Преобразование категориальных переменных в числовые
        df['Attitude'] = LabelEncoder().fit_transform(df['Attitude'])
        df['Speaker'] = LabelEncoder().fit_transform(df['Speaker'])

        # Инициализация словаря для хранения результатов
        results = {}

        # Определение предикторов
        predictors = ['Attitude', 'Speaker']

        for target in targets:
            X = df[predictors]
            y = df[target]
            # Добавление константы (intercept) для модели
            X = sm.add_constant(X)
            # Моделирование робастной регрессии
            model = sm.RLM(y, X, M=sm.robust.norms.HuberT())
            results[target] = model.fit()
        
        return results
    
    except FileNotFoundError:
        print(f"File '{data_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

def interpret_model_to_csv(results, output_path):
    if not results:
        print("No results to interpret.")
        return

    # Список для хранения строк данных
    rows = []

    for target, result in results.items():
        # Получение коэффициентов модели
        coefficients = result.params
        p_values = result.pvalues
        conf_intervals = result.conf_int()

        for param in coefficients.index:
            coef = coefficients[param]
            p_val = p_values[param]
            conf_int = conf_intervals.loc[param]

            # Запись данных в строку
            row = {
                'Target Variable': target,
                'Predictor': param,
                'Coefficient': coef,
                'P-Value': p_val,
                '95% CI Lower': conf_int[0],
                '95% CI Upper': conf_int[1],
                'Significance': 'Significant' if p_val < 0.05 else 'Not Significant'
            }
            rows.append(row)

    # Создание DataFrame и сохранение в CSV
    df_interpretation = pd.DataFrame(rows)
    df_interpretation.to_csv(output_path, index=False)

# Пример вызова функций
data_path = '01.csv'  # Убедитесь, что путь к файлу корректный
output_path = '19.csv'

results = robust_regression(data_path)
interpret_model_to_csv(results, output_path)

In [47]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.robust.robust_linear_model import RLM

# Загрузка данных
data = pd.read_csv('01.csv')

# Замена запятых на точки в целевых переменных
target_vars = ['60_65_AMP_E', '60_65_AMP_C', '60_65_diff_AMP', '50_75_AMP_E', '50_75_AMP_C', '50_75_diff_AMP']
data[target_vars] = data[target_vars].replace(',', '.', regex=True)

# Преобразование категориальных переменных
data['Attitude'] = data['Attitude'].astype('category')
data['Speaker'] = data['Speaker'].astype('category')

# Преобразование целевых переменных в числовые значения
for target in target_vars:
    data[target] = pd.to_numeric(data[target], errors='coerce')

# Преобразование предикторов в числовые значения
predictors = ['CRT', 'NFCS', 'CS', 'Persuasiveness', 'Sharing', 'Congruence', 
              'Authoritativeness', 'Trust', 'Factor 1', 'Factor 2']

for predictor in predictors:
    data[predictor] = pd.to_numeric(data[predictor], errors='coerce')

# Удаление строк с NaN
data = data.dropna()

# Список для хранения результатов
results = []

for target in target_vars:
    for predictor in predictors:
        # Создание модели
        if predictor in ['Attitude', 'Speaker']:
            # Создание фиктивных переменных для категориальных предикторов
            dummies = pd.get_dummies(data[predictor], drop_first=True)
            X = pd.concat([data[predictor] for predictor in predictors if predictor != target] + [dummies], axis=1)
        else:
            X = data[[predictor]]

        y = data[target]
        
        # Добавление константы
        X = sm.add_constant(X)

        # Робастная регрессия
        model = RLM(y, X).fit()
        
        # Сохранение результатов
        coef = model.params[predictor]
        p_value = model.pvalues[predictor]
        conf_int = model.conf_int().loc[predictor]

        formula = f"{target} = " + " + ".join([f"{coef:.4f} * {var}" for var, coef in model.params.items() if var != 'const'])

        results.append({
            'Formula': formula,
            'Dependent Variable': target,
            'Independent Variable': predictor,
            'Coefficient': coef,
            'P-Value': p_value,
            'Confidence Interval (Lower)': conf_int[0],
            'Confidence Interval (Upper)': conf_int[1]
        })

# Создание DataFrame из результатов
results_df = pd.DataFrame(results)

# Сохранение результатов в Excel
results_df.to_excel('21.xlsx', index=False)